In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
# !pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab

Using TensorFlow backend.


In [0]:
import torch
from torch.utils import data
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torchvision
from torchvision import transforms
from pathlib import Path
import PIL
import matplotlib.pyplot as plt
import numpy as np

## Using GPU for computation

In [0]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
cudnn.benchmark = True

## Mapping class names to integers

In [0]:
mappings1 = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
mappings2 = {mapping: i for i, mapping in enumerate(mappings1)}

## Defining Dataset class

In [0]:
class CINIC(Dataset):
  def __init__(self, root_dir, transform=None):
    path = Path(root_dir)
    self.img_list = []
    self.label_list = []
    self.transform = transform
    for filename in path.rglob('*.png'):
      self.img_list.append(filename)
      self.label_list.append(mappings2[str(filename).split('/')[-2]])

  def __len__(self):
    return len(self.label_list)

  def __getitem__(self, idx):
    img_file, label = self.img_list[idx], self.label_list[idx]
    img = PIL.Image.open(self.img_list[idx]).convert('RGB')
    if self.transform:
      img = self.transform(img)
    return img, label

## Defining batch parameters and normalization parameters

In [0]:
params = {'batch_size': 64,
          'shuffle': True,
          'num_workers': 4}

data_mean = [0.47889522, 0.47227842, 0.43047404]
data_std = [0.24205776, 0.23828046, 0.25874835]

## Defining transforms for all 3 sets

In [0]:
train_transform = transforms.Compose([  
    transforms.RandomResizedCrop(32),                               
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=data_mean, std=data_std)
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=data_mean, std=data_std)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=data_mean, std=data_std)
])

## Utilizing Dataloader class to read data in batches

In [0]:
train_set = CINIC(root_dir='data/CINIC-10/train/', transform=train_transform)
train_generator = data.DataLoader(train_set, **params)

val_set = CINIC(root_dir='data/CINIC-10/valid/', transform=val_transform)
val_generator = data.DataLoader(val_set, **params)

test_set = CINIC(root_dir='data/CINIC-10/test/', transform=test_transform)
test_generator = data.DataLoader(test_set, **params)

## Creating the Network

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv1_bn = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc1_bn = nn.BatchNorm1d(120)
        self.fc2 = nn.Linear(120, 84)
        self.fc2_bn = nn.BatchNorm1d(84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1_bn(self.conv1(x))))
        x = self.pool(F.relu(self.conv2_bn(self.conv2(x))))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1_bn(self.fc1(x)))
        x = F.relu(self.fc2_bn(self.fc2(x)))
        x = self.fc3(x)
        return x

net = Net()
net.to(device)
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv1_bn): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc1_bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc2_bn): BatchNorm1d(84, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


## Defining loss function and optimizer

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(net.parameters(), lr=0.001)

## Defining tensorboard object to visualize train and validation loss

In [38]:
tb = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://3ba17921.ngrok.io


In [26]:
j = 0
for epoch in range(20):

  val_running_loss = 0
  net.train()
  for i, dat in enumerate(train_generator):
    j += 1
    # get image, label pair
    inputs, labels = dat

    #using GPU
    inputs = inputs.to(device)
    labels = labels.to(device)

    # set parameter gradients to 0
    optimizer.zero_grad()

    # forward pass for a batch
    outputs = net(inputs)

    # Compute loss
    loss = criterion(outputs, labels)

    # Backpropagate
    loss.backward()

    # update the weights
    optimizer.step()

    # print training loss
    #print('[{}, {}] training loss:{}'.format(epoch+1, i+1, loss.item()))
    tb.save_value('Train Loss', 'train_loss', j, loss.item())

  net.eval()
  with torch.no_grad():
    for k, dat1 in enumerate(val_generator):
      inputs1, labels1 = dat1
      inputs1 = inputs1.to(device)
      labels1 = labels1.to(device)
      outputs1 = net(inputs1)
      loss1 = criterion(outputs1, labels1)
      val_running_loss += loss1

    avg_val_loss = float(val_running_loss)/(k+1)
    print('epoch {}, validation loss: {}'.format(epoch+1, avg_val_loss))
    tb.save_value('Validation Loss', 'val_loss', epoch+1, avg_val_loss)

print('Finished Training')

epoch 1, validation loss: 1.4255568037324537
epoch 2, validation loss: 1.3582773215396233
epoch 3, validation loss: 1.3171266990938166
epoch 4, validation loss: 1.2540081053993426
epoch 5, validation loss: 1.2697775596070984
epoch 6, validation loss: 1.2250465550317609
epoch 7, validation loss: 1.2156548625455312
epoch 8, validation loss: 1.221214744497379
epoch 9, validation loss: 1.2174939858464375
epoch 10, validation loss: 1.20006222375844
epoch 11, validation loss: 1.1969650040811568
epoch 12, validation loss: 1.2072135312555525
epoch 13, validation loss: 1.1933394203646943
epoch 14, validation loss: 1.193514760877532
epoch 15, validation loss: 1.1923298025775142
epoch 16, validation loss: 1.1891987467794953
epoch 17, validation loss: 1.1778091615027542
epoch 18, validation loss: 1.1897228596137615
epoch 19, validation loss: 1.1795961424740138
epoch 20, validation loss: 1.1809912317597282
Finished Training


## Overall test set accuracy

In [29]:
correct = 0
total = 0
with torch.no_grad():
  for data in test_generator:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Test set accuracy = {}%'.format(100.0 * correct / total))

Test set accuracy = 57.83555555555556%


## Class by class test accuracy

In [0]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in test_generator:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

In [31]:
for i in range(10):
  print('Accuracy of {}: {}%'.format(mappings1[i], round(100 * class_correct[i] / class_total[i], 2)))

Accuracy of airplane: 70.45%
Accuracy of automobile: 60.63%
Accuracy of bird: 47.37%
Accuracy of cat: 44.09%
Accuracy of deer: 51.06%
Accuracy of dog: 39.51%
Accuracy of frog: 64.85%
Accuracy of horse: 66.14%
Accuracy of ship: 67.88%
Accuracy of truck: 60.77%


## Confusion Matrix

In [32]:
confusion_matrix = torch.zeros(10, 10)
with torch.no_grad():
  for data in test_generator:
    inputs, labels = data
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = net(inputs)
    _, predicted = torch.max(outputs, 1)
    for t, p in zip(labels.view(-1), predicted.view(-1)):
      confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)

tensor([[6299.,  243.,  470.,  109.,  159.,  136.,   43.,  222., 1000.,  319.],
        [ 311., 5796.,  112.,  148.,  130.,  184.,   66.,  164.,  641., 1448.],
        [ 588.,   57., 4270.,  904., 1023.,  763.,  655.,  313.,  371.,   56.],
        [ 136.,   71.,  600., 3955., 1011., 1829.,  713.,  360.,  218.,  107.],
        [ 215.,   62.,  560.,  900., 4670.,  880.,  313., 1080.,  235.,   85.],
        [ 142.,  105.,  603., 1724., 1180., 3794.,  299.,  811.,  215.,  127.],
        [  60.,   54.,  579., 1191.,  484.,  446., 5992.,   67.,  100.,   27.],
        [ 146.,   82.,  242.,  433.,  964.,  879.,   50., 5886.,  147.,  171.],
        [ 948.,  350.,  336.,  279.,  213.,  178.,   81.,  166., 5989.,  460.],
        [ 360., 1606.,   89.,  180.,  148.,  170.,   43.,  293.,  710., 5401.]])


In [0]:
# !mkdir /content/data
# !cp /content/drive/My\ Drive/csci_677_hw4/CINIC-10.tar.gz /content/data
# !mkdir /content/data/CINIC-10
# !tar -xvzf /content/data/CINIC-10.tar.gz -C /content/data/CINIC-10